## Imports

In [2]:
import numpy as np

from pylabnet.utils.logging.logger import LogClient
from pylabnet.utils.helper_methods import load_config
import pylabnet.utils.pulseblock.pulse as po
import pylabnet.utils.pulseblock.pulse_block as pb
from pylabnet.utils.pulseblock.pb_iplot import iplot
from pylabnet.utils.pulseblock.pb_sample import pb_sample
from pylabnet.hardware.awg.zi_hdawg import Driver, Sequence, AWGModule
from pylabnet.hardware.staticline import staticline
from pylabnet.utils.zi_hdawg_pulseblock_handler.zi_hdawg_pb_handler import DIOPulseBlockHandler
from pylabnet.network.client_server import dio_breakout
from pylabnet.network.client_server import si_tt

from pyvisa import VisaIOError, ResourceManager


from pylabnet.gui.igui.iplot import SingleTraceFig, MultiTraceFig, StaggeredTraceFig


# Digital pulse playback using `pulseblock` and the DIO output of a ZI HDWAG

In [3]:
dev_id = 'dev8227'

# Instantiate logger.
logger = LogClient(
    host='140.247.189.94',
    port=1491,
    module_tag=f'ZI HDAWG {dev_id}'
)

# Instanciate HDAWG driver.
hd = Driver(dev_id, logger=logger)

Let's start with defining the digital part of a typical rabi-sequence. We use the `rabi_element` function as defined in the `pulseblock` demo notebook:

In [4]:
# Can be used to slow things down
scaling = 1

# Define the pulse sequence
test_pulse = pb.PulseBlock(
    p_obj_list=[
        po.PTrue(ch='toptica', dur=7e-6*scaling),
        po.PTrue(ch='gate', t0=1.9e-6*scaling, dur=0.1e-6*scaling)
    ]
)

# Add some fake pulses in the "readout" window
count_t0s = np.linspace(2.1*10**-6, 6.9*10**-6, 10)*scaling
for count_t0 in count_t0s:
    test_pulse.insert(
        po.PTrue(
            ch='fake_counts',
            dur=50*10**-9*scaling,
            t0=count_t0
        )
    )

# Default states (in future, module should auto default to False)
test_pulse.dflt_dict = dict(
    gate=po.DFalse(),
    toptica=po.DFalse(),
    fake_counts=po.DFalse()
)

# Plot
iplot(test_pulse)

Now, let's turn this `pulseblock` instance into an intruction set for the DIO outputs of the HDAWG. We want to get a series on `setDIO()` and `wait()` commands, which reproduce the pulse sequence.

In [5]:
# Load and verify assignment dict
assignment_dict = load_config('dio_assignment', logger=logger)
print(assignment_dict)

{'green': 31, 'toptica': 30, 'gate': 29, 'fake_counts': 28}


In [6]:
# Instanciate pulseblock handler.
pb_handler = DIOPulseBlockHandler(
    pb = test_pulse,
    assignment_dict=assignment_dict,
    hd=hd
)

In [7]:
# Generate .seqc instruction set which represents pulse sequence.
dig_pulse_sequence = pb_handler.get_dio_sequence()

In [8]:
# Let's have a look.
print(dig_pulse_sequence)

setDIO(1073741824);wait(564);setDIO(1610612736);wait(27);setDIO(1073741824);wait(25);setDIO(1342177280);wait(12);setDIO(1073741824);wait(140);setDIO(1342177280);wait(12);setDIO(1073741824);wait(140);setDIO(1342177280);wait(12);setDIO(1073741824);wait(140);setDIO(1342177280);wait(12);setDIO(1073741824);wait(140);setDIO(1342177280);wait(13);setDIO(1073741824);wait(139);setDIO(1342177280);wait(12);setDIO(1073741824);wait(140);setDIO(1342177280);wait(13);setDIO(1073741824);wait(139);setDIO(1342177280);wait(13);setDIO(1073741824);wait(139);setDIO(1342177280);wait(12);setDIO(1073741824);wait(140);setDIO(1342177280);wait(13);setDIO(1073741824);wait(9);setDIO(0);wait(0);


This `.seqc` snippet can now be used in any sequence to reproduce the pulse sequence above. Let's try to write a simple sequence which outputs the digital pulse sequence as well as a trigger signal, and let's look at the results on the scope. 

To do so, connect marker output 1 of the HDWAG to the scope and use it as scope trigger, end then connect DIO pins 15, 17, and 31 each to one scope channel.

In [9]:
# This sequence will send out a trigger and execute the commands  dig_pulse (yet to be specified).
sequence_txt = """\

        while (1) {

          _dig_pulse_
          
          
          // Wait
          wait(1000);

        }
        """

# Create Sequence instance.
seq = Sequence(hd, sequence_txt, ['dig_pulse'])

# Replace dig_pulse placeholder by digital pulse sequence instruction set.
seq.replace_placeholder('dig_pulse', dig_pulse_sequence)

In [10]:
# Create an instance of the AWG Module.
awg = AWGModule(hd, 3)
awg.set_sampling_rate('2.4 GHz') # Set 2.4 GHz sampling rate.

# Upload sequence.
if awg is not None:
    awg.compile_upload_sequence(seq)

In [11]:
# Now we're almost ready, we only have to make sure that the 8 bit buses for bits 15, 17, and 31 are driven. 
# This can be done automatically by calling the following:

pb_handler.setup_hd()

## DIO breakout config

In [10]:
# Connect to DIO breakout server (launched via launch control)
dio_ip = '140.247.189.82'
dio_port = 30508
dio_breakout_client = dio_breakout.Client(
    host = dio_ip,
    port = dio_port
)

In [12]:
# target_voltage = 0.3
# dio_breakout_client.set_high_voltage(1, 3, target_voltage)

In [11]:
for channel in range(4):
    print(f'Channel {channel}, low voltage: {dio_breakout_client.get_low_voltage(1, channel)}, '
          f'high voltage: {dio_breakout_client.get_high_voltage(1, channel)}')

Channel 0, low voltage: 0.0, high voltage: 5.0000762951094835
Channel 1, low voltage: 0.0, high voltage: 5.0000762951094835
Channel 2, low voltage: 0.0, high voltage: 5.0000762951094835
Channel 3, low voltage: 0.0, high voltage: 0.29999237048905164


In [12]:
for channel in range(4):
    print(f'Channel {channel}, low voltage: {dio_breakout_client.get_low_voltage(0, channel)}, '
          f'high voltage: {dio_breakout_client.get_high_voltage(0, channel)}')

Channel 0, low voltage: 0.0, high voltage: 4.0
Channel 1, low voltage: 0.0, high voltage: 5.0000762951094835
Channel 2, low voltage: 0.0, high voltage: 5.0000762951094835
Channel 3, low voltage: 0.0, high voltage: 4.0


In [121]:
target_voltage = 3
dio_breakout_client.set_high_voltage(0, 0, target_voltage)

0

## Setup histogram

In [13]:
ctr = si_tt.Client(
    host='140.247.189.82',
    port=42135
)

In [14]:
h_name = 'histogram'
ctr.start_histogram(
    name=h_name,
    start_ch=2,
    click_ch=1,
    binwidth=1000,
    n_bins=10000
)

In [32]:
ctr.get_counts(name=h_name)[0]

array([      0,       0,       0,       0,       0,       0,       0,
             0,       0,       0,       0,       0,       0,       0,
             0,       0,       0,       0,       0,       0,       0,
             0,       0,       0,       0,       0,       0,       0,
             0,       0,       0,       0,       0,       0,       0,
             0,       0,       0,       0,       0,       0,       0,
             0,       0,       0,       0,       0,       0,       0,
             0,       0,       0,       0,       0,       0,       0,
             0,       0,       0,       0,       0,       0,       0,
             0,       0,       0,       0,       0,       0,       0,
             0,       0,       0,       0,       0,       0,       0,
             0,       0,       0,       0,       0,       0,       0,
             0,       0,       0,       0,       0,       0,       0,
             0,       0,       0,       0,       0,       0,       0,
             0,     

## Setup gated counter

In [11]:
from pylabnet.network.client_server import si_tt_cnt_monitor

In [12]:
ctr = si_tt_cnt_monitor.Client(
    host='140.247.189.82',
    port=4616
)

In [13]:
c_name = 'gated_ctr'
ctr.set_channels(name=c_name, ch_list=[1], gates=[2])

In [14]:
ctr.setup_gated_counter(name=c_name)

In [17]:
ctr.get_counts(name=c_name)

array([10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10

## Run AWG

In [17]:
# Now we're ready, start the Start the AWG
awg.start()

In [ ]:
# TODO: TEST COUNTERS

In [29]:
awg.stop()

# Staticline stuff

In [3]:

snspd1 = staticline.Driver(
    name='snspd1',
    logger=logger,
    hardware_module=hd,
    DIO_bit=24,
) 


snspd2 = staticline.Driver(
    name='snspd1',
    logger=logger,
    hardware_module=hd,
    DIO_bit=27,
) 

'dev8227'

In [37]:
snspd1.up()

In [16]:
snspd1.down()

In [177]:
snspd2.up()

In [17]:
snspd2.down()

In [13]:
target_voltage = 0
dio_breakout_client.set_high_voltage(0, 0, target_voltage)

0

In [14]:
target_voltage = 0
dio_breakout_client.set_high_voltage(0, 3, target_voltage)

0

In [15]:
dio_breakout_client.get_high_voltage(0, 0)

0.0

In [ ]:

scope = Client(
    host='localhost',
    port=2253
)

In [8]:
# Let assume we have a shutter connected to the HDAWG DIO-pin 1 which 
# is open if the pin voltage is high and closed otherwise.

aom = staticline.Driver(
    name='AOM',
    logger=logger,
    hardware_module=hd,
    DIO_bit=30,
) 

In [11]:
aom.up()


In [12]:
aom.down()

In [16]:
# AOM Switch 
import pylabnet.hardware.ni_daqs.nidaqmx_card as nidaqmx
import pylabnet.network.client_server.nidaqmx_card as nidaqmx_card_server

In [23]:
aom_daq = nidaqmx_card_server.Client(
    host='140.247.189.82',
    port=38676
)

In [24]:
aom_daq.set_ao_voltage('ao2', 0)

In [55]:
aom_daq.set_ao_voltage('ao2', 0.05)

In [54]:
aom.down()

In [56]:
aom.up()

In [188]:
daq.set_ao_voltage('ao0',6.92)

In [92]:
aom_daq.set_ao_voltage('ao3',0)

In [77]:
import pylabnet.network.client_server.thorlabs_pm320e as pm

In [80]:
powermeter = pm.Client(
    host = "140.247.189.94",
    port=4373
)

In [82]:
powermeter.get_power(0)

pyvisa.errors.VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.

========= Remote Traceback (1) =========
Traceback (most recent call last):
  File "C:\Users\Roger\pylabnet\venv\dev\lib\site-packages\rpyc-4.1.5-py3.6.egg\rpyc\core\protocol.py", line 320, in _dispatch_request
    res = self._HANDLERS[handler](self, *args)
  File "C:\Users\Roger\pylabnet\venv\dev\lib\site-packages\rpyc-4.1.5-py3.6.egg\rpyc\core\protocol.py", line 593, in _handle_call
    return obj(*args, **dict(kwargs))
  File "C:\Users\Roger\pylabnet\pylabnet\network\client_server\thorlabs_pm320e.py", line 7, in exposed_get_power
    return self._module.get_power(channel)
  File "C:\Users\Roger\pylabnet\pylabnet\hardware\power_meter\thorlabs_pm320e.py", line 41, in get_power
    power = self.device.query(f':POW{channel}:VAL?')
  File "C:\Users\Roger\pylabnet\venv\dev\lib\site-packages\pyvisa-1.10.1-py3.6.egg\pyvisa\resources\messagebased.py", line 613, in query
    return self.read()


In [84]:
import numpy as np

num_sweeps = 10
voltages = np.linspace(0, 2, num_sweeps)
powers = np.zeros_like((voltages))

for i, voltage in enumerate(voltages):
    power = 1
    powers[i] = power